# **Analysis of Heart Disease Mortality Data**

#### **Samarth Tuli, Ndenko Fontem, Yi Zhu, Kimia Samieinejad**    


### **Importance of Heart Disease Mortality Research**    
People across the world have experiencing several physical diseases ranging from diabetes to arthritis and mental health disorders from depression to suicide within the past 2 decades, which has made global health a significant priority of the research community. However, one of the most important disease types that is being studied is heart disease, especially coronary (ischemic) artery disease. According to the [World Health Organization](https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1), heart disease is the leading global cause of death, which results in 17.9 million annual deaths. 4 out of every 5 heart disease deaths are caused by heart attacks and strokes, which can be caused by any mental or physical health factors, and one third of heart disease deaths occur prematurely to people under the age of 70, which means everyone is vulnerable to heart disease. Research efforts led by NIH and WHO have shown that a wide variety of behavioral, medical, and socioeconomic risk factors can be underlying causes for heart disease mortality including tobacco and alcohol consumption, obesity, lack of physical activity, malnutrition, increased blood pressure, and restricted access to primary healthcare facilities. Other commorbidities (pre-existing conditions) such as diabetes, arthritis, chronic kidney disease, and anxiety problems. Given the massive amount of data collected on these factors and how heart disease deaths can vary for each country's population, data scientists have a significant role to break down this data into insights that can guide the future path of heart disease mortality research. In our dataset, we are defining heart disease mortality rate as the number of heart disease related deaths per 100,000 people and will cover it for all countries from 2012-2017 and will focus upon countries within North America and Europe.

### **What is coronary heart disease?**

According to the NIH's latest research, [coronary heart disease](https://www.nhlbi.nih.gov/health/coronary-heart-disease) is a cardiovascular disease where the arteries cannot provide sufficient oxygen to a person's bloodstream. The primary cause of CHD is high cholesterol forming plaque along the lining of the arteries, which can constrict blood flow, cause blood vessels to stop functioning normally, and also increase the chances of severe chest pain or heart attacks or strokes or cardiac arrest. Although the risk of coronary heart disease can be reduced through lifestyle changes, many people don't take immediate action. This has resulted in it becoming globally widespread with 650000 deaths per year due to generalized heart disease, 11% of adults being diagnosed with heart disease, and 366000 annual deaths due to CHD specifically in the US alone. This background clearly demonstrates that heart disease mortality data needs to be analyzed by data scientists to provide key insights that will mitigate risk for future heart patients.

### **Tutorial Purpose**

The objective of this tutorial is to evaluate many factors that may positively or negatively affect the heart disease mortality rates of populations across different countries so that we can get a better understanding of what factors should be focused upon most by the research community to reduce the overall risk of heart disease-related deaths in North America and European countries. Data science is the right tool to achieve this because it will allow us to deconstruct complex heart disease mortality data into specific insights and recommendations that can be used by heart disease research leaders and policymakers to take immediate action through a 5-stage pipeline: data collection and processing, exploratory data analysis and visualizations, analysis/hypothesis testing/use of ML models, and insights & policy decisions.

## **Data Collection and Processing**

We collected data from the Global Health Observatory database on the [World Health Organization](https://www.who.int/data/gho). This specialized agency established by the United Nations is responsible for harmonizing the global health activties and aiding governments around the world in inhancing their healthcare systems. The specific input features we got data for from WHO for each country and year were age-standardized suicide rate (per 100000 people), mean BMI, raised blood pressure, mean HDL cholesterol, meany systolic blood pressure, health expenditure as % of GDP, and % of overweight adults. We also collected data from [Our World in Data](https://ourworldindata.org/). We got data on heart disease mortality rate (deaths per 100,000 people), which is our target feature we're trying to predict. We also retrieved two more input features, which are deaths caused by type 1 and 2 diabetes in each country.  Our World is a online scientific publication that focuses on global challenges such as poverty, disease, and inequality. We collected data for all 183 countries in 2012-2017 for all 9 features (including heart disease mortality rate) from WHO and Our World in Data. However, two issues we encountered for our datasets are that some of them do not contain data for all 183 countries or our designated year range (2012-2017) in each feature. To combat this problem, we performed imputation via linear regression to predict and fill in the missing values for each input feature.

In [416]:
# packages for data collection and processing
import numpy as np
import pandas as pd

# packages for plotting graphs (feature visualization)
import matplotlib.pyplot as plt

# packages for ML and filling in missing values
from sklearn import linear_model

### **Add heart disease mortality (death rate) data**
The first step we will take is to import the [heart disease death rate](https://ourworldindata.org/grapher/cardiovascular-disease-death-rates) (target feature) data and remove the columns we won't use. This feature is an age-standardized estimate for both sexes and has data from 1990-2019 but we will only be using the rows from 2012-2017 in this table.

In [417]:
# Import and process age-standardized heart disease death rates data for both sexes (# of heart disease deaths per 100000 people)
heart_disease_death_rates = pd.read_csv('data/cardiovascular-disease-death-rates.csv')
heart_disease_death_rates = heart_disease_death_rates[['Entity', 'Year', 'Deaths - Cardiovascular diseases - Sex: Both - Age: Age-standardized (Rate)']]
heart_disease_death_rates = heart_disease_death_rates.rename(columns = {"Entity":"Country", 'Deaths - Cardiovascular diseases - Sex: Both - Age: Age-standardized (Rate)':'Heart Disease Mortality Rate (Deaths per 100K people)'})
heart_disease_death_rates = heart_disease_death_rates[(heart_disease_death_rates['Year']>=2012) & (heart_disease_death_rates['Year']<=2017)]
heart_disease_death_rates = heart_disease_death_rates.reset_index(drop = True)
percentage_missing_values_heart_disease_mortality = ((float(heart_disease_death_rates['Heart Disease Mortality Rate (Deaths per 100K people)'].isna().sum()))/heart_disease_death_rates['Heart Disease Mortality Rate (Deaths per 100K people)'].count()) * 100.0
print("Percentage Missing Values: ", percentage_missing_values_heart_disease_mortality)
heart_disease_death_rates

Percentage Missing Values:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people)
0,Afghanistan,2012,618.45
1,Afghanistan,2013,607.69
2,Afghanistan,2014,600.55
3,Afghanistan,2015,597.02
4,Afghanistan,2016,591.59
...,...,...,...
1363,Zimbabwe,2013,331.36
1364,Zimbabwe,2014,330.58
1365,Zimbabwe,2015,329.57
1366,Zimbabwe,2016,327.64


### **Add anxiety disorder prevalence data**
The second step we will take is to import the [anxiety disorder prevalence data](https://ourworldindata.org/grapher/anxiety-disorders-prevalence?tab=chart) and remove the columns we aren't going to use. This feature is an age-standardized estimate and is a % of each country's population that comes from Our World in Data. The anxiety prevalence table includes data from 2012-2017 for both sexes. There are no missing values in the anxiety data before joining, so no imputation is required. However, after merging the anxiety data with the heart disease mortality data frame, we can see that 6.5% of the anxiety data has missing values in the main data frame. Therefore, we have to fill each missing value with the average anxiety prevalence for that particular year. After doing so, there are no more missing values.

In [418]:
# Import and process % of population with anxiety disorders data for both sexes
# Extract 3 fields (country, year, and anxiety disorder prevalence
anxiety_prevalence_all_countries = pd.read_csv('data/anxiety-disorders-prevalence.csv')
anxiety_prevalence_all_countries = anxiety_prevalence_all_countries[['Entity', 'Year', 'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized']]
anxiety_prevalence_all_countries = anxiety_prevalence_all_countries[(anxiety_prevalence_all_countries['Year']>=2012) & (anxiety_prevalence_all_countries['Year']<=2017)]
anxiety_prevalence_all_countries = anxiety_prevalence_all_countries.rename(columns = {'Entity':'Country', 'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized':"% of population with anxiety disorders"})
anxiety_prevalence_all_countries = anxiety_prevalence_all_countries.sort_values(by = ['Country', 'Year'])
anxiety_prevalence_all_countries = anxiety_prevalence_all_countries.reset_index(drop = True)
# Compute % of missing values in anxiety disorder data
percentage_missing_values_anxiety_before = ((float(anxiety_prevalence_all_countries['% of population with anxiety disorders'].isna().sum()))/anxiety_prevalence_all_countries['% of population with anxiety disorders'].count()) * 100.0
print("Percent Missing Anxiety Values Before Join: ", percentage_missing_values_anxiety_before)
# Substituting all NaN values in anxiety col of main dataframe with mean values of corresponding years
heart_disease_death_rates['% of population with anxiety disorders'] = anxiety_prevalence_all_countries['% of population with anxiety disorders']
heart_disease_death_rates['% of population with anxiety disorders'] = heart_disease_death_rates['% of population with anxiety disorders'].fillna(0.0)
for index, row in heart_disease_death_rates.iterrows():
    if row['% of population with anxiety disorders'] == 0.0:
        anxiety_col = heart_disease_death_rates[heart_disease_death_rates['Year'] == row['Year']]['% of population with anxiety disorders']
        mean_anxiety = anxiety_col.mean(axis = 0)
        heart_disease_death_rates.loc[index, '% of population with anxiety disorders'] = mean_anxiety
# Compute % of missing values in anxiety after join
percentage_missing_values_anxiety_after = ((float(heart_disease_death_rates['% of population with anxiety disorders'].isna().sum()))/heart_disease_death_rates['% of population with anxiety disorders'].count()) * 100.0
print("Percent Missing Anxiety Values After Join and Imputation: ", percentage_missing_values_anxiety_after)
# Outputting heart disease death rates df with anxiety column
heart_disease_death_rates

Percent Missing Anxiety Values Before Join:  0.0
Percent Missing Anxiety Values After Join and Imputation:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders
0,Afghanistan,2012,618.45,4.739260
1,Afghanistan,2013,607.69,4.766758
2,Afghanistan,2014,600.55,4.793179
3,Afghanistan,2015,597.02,4.810425
4,Afghanistan,2016,591.59,4.815518
...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508
1364,Zimbabwe,2014,330.58,4.119858
1365,Zimbabwe,2015,329.57,4.124367
1366,Zimbabwe,2016,327.64,4.126087


### **Add BMI data**

The third step we will take is to import the [mean BMI data](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/mean-bmi-(kg-m-)-(age-standardized-estimate)) and remove the columns we aren't going to use. This feature data comes from the World Health Organization. The mean BMI table includes data from 2012-2017 that separates BMI  by sex. We will only consider the mean BMI for both sexes combined (is an average of both sexes separately). There are 2.09% missing values in the BMI data before joining. After merging the BMI data with the heart disease mortality data frame, we can see that 43.25% of the BMI data has missing values in the main data frame. Therefore, we have to perform imputation by filling each missing value with the average mean BMI for that particular year. After doing so, there are no more missing values. However, this might introduce some bias.

In [419]:
# Import and process mean BMI data for both sexes
bmi_all_countries = pd.read_csv('data/BMI.csv')
bmi_all_countries = bmi_all_countries[['Location', 'Period', 'Dim1','FactValueNumeric']]
bmi_all_countries = bmi_all_countries.rename(columns = {"Location": "Country", "Period": "Year", "Dim1":"Sex", "Dim2":"Age", "FactValueNumeric": "Mean BMI"})
bmi_all_countries = bmi_all_countries[bmi_all_countries["Sex"] == "Both sexes"]
# Sort data frame by country and year and reset index
bmi_all_countries = bmi_all_countries.sort_values(by=['Country', 'Year'])
bmi_all_countries = bmi_all_countries.reset_index(drop = True)
bmi_all_countries = bmi_all_countries[['Country', 'Year', 'Mean BMI']]
# Compute % of missing values in BMI data frame before join
percentage_missing_values_bmi_before = ((float(bmi_all_countries['Mean BMI'].isna().sum()))/bmi_all_countries['Mean BMI'].count()) * 100.0
print("Percentage of Missing Values for Mean BMI Before Join: ", percentage_missing_values_bmi_before)
# Merge BMI data with heart disease mortality data frame, Substituting all NaN values in BMI col of main dataframe with mean values of corresponding years
heart_disease_death_rates['Mean BMI'] = bmi_all_countries['Mean BMI']
heart_disease_death_rates['Mean BMI'] = heart_disease_death_rates['Mean BMI'].fillna(0.0)
for index, row in heart_disease_death_rates.iterrows():
    if row['Mean BMI'] == 0.0:
        bmi_col = heart_disease_death_rates[heart_disease_death_rates['Year'] == row['Year']]['Mean BMI'].dropna()
        bmi_mean = bmi_col.mean(axis = 0)
        heart_disease_death_rates.loc[index, 'Mean BMI'] = bmi_mean
# Compute % of missing values in BMI after join
percentage_missing_values_bmi_after = ((float(heart_disease_death_rates['Mean BMI'].isna().sum()))/heart_disease_death_rates['Mean BMI'].count()) * 100.0
print("Percentage of Missing Values in Mean BMI After Join: ", percentage_missing_values_bmi_after)
# Outputting heart disease death rates df with BMI column
heart_disease_death_rates

Percentage of Missing Values for Mean BMI Before Join:  2.094240837696335


Percentage of Missing Values in Mean BMI After Join:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI
0,Afghanistan,2012,618.45,4.739260,22.900000
1,Afghanistan,2013,607.69,4.766758,23.000000
2,Afghanistan,2014,600.55,4.793179,23.200000
3,Afghanistan,2015,597.02,4.810425,23.300000
4,Afghanistan,2016,591.59,4.815518,23.400000
...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756
1364,Zimbabwe,2014,330.58,4.119858,24.120932
1365,Zimbabwe,2015,329.57,4.124367,24.157551
1366,Zimbabwe,2016,327.64,4.126087,24.128846


### **Add mean HDL cholesterol data**

The fourth step we will take is to import the [mean HDL cholesterol data](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/gho-mean-hdl-cholesterol-crude) and remove the columns we aren't going to use. This feature data comes from the World Health Organization. The mean HDL cholesterol table includes data from 2012-2017 that separates mean HDL cholesterol by sex. We will only consider the mean HDL cholesterol for both sexes combined (is an average of both sexes separately). There are no missing values in the mean HDL cholesterol data before joining, so no initial imputation is required. However, after merging the cholesterol data with the heart disease mortality data frame, we can see that 19.37% of the cholesterol data has missing values in the main data frame. Therefore, we have to fill each missing value with the average mean HDL cholesterol for that particular year. After doing so, there are no more missing values but we must keep in mind that some bias might be introduced.

In [420]:
# Import mean HDL cholesterol data of both sexes
mean_HDL_cholesterol = pd.read_csv("data/cholestrol2012-2017.csv")
mean_HDL_cholesterol = mean_HDL_cholesterol[['Location', 'Period', 'Dim1','FactValueNumeric']]
mean_HDL_cholesterol = mean_HDL_cholesterol.rename(columns = {"Location": "Country", "Period": "Year", "Dim1":"Sex", "Dim2":"Age", "FactValueNumeric": "Cholesterol Level"})
mean_HDL_cholesterol = mean_HDL_cholesterol[mean_HDL_cholesterol["Sex"] == "Both sexes"]
# Sort data frame by country and year and reset index
mean_HDL_cholesterol = mean_HDL_cholesterol.sort_values(by=['Country', 'Year'])
mean_HDL_cholesterol = mean_HDL_cholesterol.reset_index(drop = True)
mean_HDL_cholesterol  = mean_HDL_cholesterol[['Country', 'Year', 'Cholesterol Level']]
# Compute % of missing values in cholesterol data frame before join
percentage_missing_values_cholesterol_before = ((float(mean_HDL_cholesterol['Cholesterol Level'].isna().sum()))/mean_HDL_cholesterol['Cholesterol Level'].count()) * 100.0
print("Percentage of Missing Values for Mean HDL Cholesterol Before Join: ", percentage_missing_values_cholesterol_before)
# Merge cholesteorl data with heart disease mortality data frame, Substituting all NaN values in mean HDL cholesterol col of main dataframe with mean values of corresponding years
heart_disease_death_rates['Mean HDL Cholesterol'] = mean_HDL_cholesterol['Cholesterol Level']
heart_disease_death_rates['Mean HDL Cholesterol'] = heart_disease_death_rates['Mean HDL Cholesterol'].fillna(0.0)
for index, row in heart_disease_death_rates.iterrows():
    if row['Mean HDL Cholesterol'] == 0.0:
        hdl_cholesterol_col = heart_disease_death_rates[heart_disease_death_rates['Year'] == row['Year']]['Mean HDL Cholesterol'].dropna()
        hdl_cholesterol_mean = hdl_cholesterol_col.mean(axis = 0)
        heart_disease_death_rates.loc[index, 'Mean HDL Cholesterol'] = hdl_cholesterol_mean
# Compute % of missing values in mean HDL cholesterol after join
percentage_missing_values_cholesterol_after = ((float(heart_disease_death_rates['Mean HDL Cholesterol'].isna().sum()))/heart_disease_death_rates['Mean HDL Cholesterol'].count()) * 100.0
print("Percentage of Missing Values in Cholesterol Level After Join: ", percentage_missing_values_cholesterol_after)
# Outputting heart disease death rates df with mean HDL cholesterol column
heart_disease_death_rates

Percentage of Missing Values for Mean HDL Cholesterol Before Join:  0.0
Percentage of Missing Values in Cholesterol Level After Join:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI,Mean HDL Cholesterol
0,Afghanistan,2012,618.45,4.739260,22.900000,1.100000
1,Afghanistan,2013,607.69,4.766758,23.000000,1.100000
2,Afghanistan,2014,600.55,4.793179,23.200000,1.100000
3,Afghanistan,2015,597.02,4.810425,23.300000,1.100000
4,Afghanistan,2016,591.59,4.815518,23.400000,1.100000
...,...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756,1.169098
1364,Zimbabwe,2014,330.58,4.119858,24.120932,1.169612
1365,Zimbabwe,2015,329.57,4.124367,24.157551,1.168585
1366,Zimbabwe,2016,327.64,4.126087,24.128846,1.171152


### **Add low physical activity DALYs data**

The fifth step we will take is to import the [low physical activity DALYs data](https://ourworldindata.org/grapher/disease-burden-by-risk-factor) and remove the columns we aren't going to use. This feature data comes from the World Health Organization. The low physical activity DALYs table includes data from 2012-2017 that separates low physical activity by sex. We will only consider the low physical activity for both sexes combined (is an average of both sexes separately). There are no missing values in the low physical activity DALYs data before joining, so no initial imputation is required. After merging the low physical activity data with the heart disease mortality data frame, we can see that there are no missing values in the low physical activity column of the main data frame. Thus, no further imputation is required.

In [421]:
# Import low physical activity DALYs data, select and rename relevant columns, and choose all rows with years 2012-2017 (will only consider data for both sexes)
low_phys_activity = pd.read_csv("data/disease-burden-by-risk-factor.csv")
low_phys_activity = low_phys_activity[['Entity', 'Year', 'DALYs (Disability-Adjusted Life Years) - Cause: All causes - Risk: Low physical activity - Sex: Both - Age: All Ages (Number)']]
low_phys_activity = low_phys_activity.loc[low_phys_activity['Year'].isin([2012,2013,2014,2015,2016,2017])]
low_phys_activity = low_phys_activity.rename(columns = {"Entity": "Country",  "DALYs (Disability-Adjusted Life Years) - Cause: All causes - Risk: Low physical activity - Sex: Both - Age: All Ages (Number)": "Low Physical Activity DALYs"})
# Sort low physical activity DALYs data by country and year and reset indexes and choose only country, year, and low physical activity DALYs columns
low_phys_activity = low_phys_activity.sort_values(by=['Country', 'Year'])
low_phys_activity.insert(2, 'Sex', 'Both sexes')
low_phys_activity = low_phys_activity.reset_index(drop = True)
low_phys_activity = low_phys_activity[['Country', 'Year', 'Low Physical Activity DALYs']]
# Compute % of missing values in low physical activity data frame before join
percentage_missing_values_low_physical_activity_before = ((float(low_phys_activity['Low Physical Activity DALYs'].isna().sum()))/low_phys_activity['Low Physical Activity DALYs'].count()) * 100.0
print("Percentage of Missing Values for Low Physical Activity DALYs Before Join: ", percentage_missing_values_cholesterol_before)
# Merge low physical activity DALY data with heart disease mortality data frame and compute % of missing values in low physical activity after join with main dataframe (no imputation required since 0% values missing after join)
heart_disease_death_rates['Low Physical Activity DALYs'] = low_phys_activity['Low Physical Activity DALYs']
percentage_missing_values_low_physical_activity_after = ((float(heart_disease_death_rates['Low Physical Activity DALYs'].isna().sum()))/heart_disease_death_rates['Low Physical Activity DALYs'].count()) * 100.0
print("Percentage of Missing Values in Low Physical Activity DALYs Level After Join: ", percentage_missing_values_low_physical_activity_after)
# Outputting heart disease death rates data frame after 
heart_disease_death_rates

Percentage of Missing Values for Low Physical Activity DALYs Before Join:  0.0
Percentage of Missing Values in Low Physical Activity DALYs Level After Join:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI,Mean HDL Cholesterol,Low Physical Activity DALYs
0,Afghanistan,2012,618.45,4.739260,22.900000,1.100000,87161.46
1,Afghanistan,2013,607.69,4.766758,23.000000,1.100000,89240.38
2,Afghanistan,2014,600.55,4.793179,23.200000,1.100000,91722.71
3,Afghanistan,2015,597.02,4.810425,23.300000,1.100000,94787.74
4,Afghanistan,2016,591.59,4.815518,23.400000,1.100000,97995.82
...,...,...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756,1.169098,6607.48
1364,Zimbabwe,2014,330.58,4.119858,24.120932,1.169612,6911.81
1365,Zimbabwe,2015,329.57,4.124367,24.157551,1.168585,7196.88
1366,Zimbabwe,2016,327.64,4.126087,24.128846,1.171152,7424.67


### **Add spirits consumption data**

The sixth step we will take is to import the [spirits consumption data](https://ourworldindata.org/grapher/spirits-consumption-per-person), which relates to alcohol consumption, and remove the columns we aren't going to use. This feature data comes from Our World in Data. The spirits consumption per person table includes data from 2012-2017 that has spirits consumption rates for both sexes combined. There are no missing values in the spirits consumption data before joining, so no initial imputation is required. After merging the spirits consumption data with the heart disease mortality data frame, we can see that there are 21.28% missing values in the spirits consumption column of the main data frame. Therefore, we have to fill each missing value with the average spirits consumption for that particular year. After doing so, there are no more missing values but we must keep in mind that some bias might be introduced.

In [422]:
# Import spirits consumption data, select and rename relevant columns, and choose all rows with years 2012-2017 (will only consider data for both sexes)
spirits_consumption = pd.read_csv("data/spirits-consumption-per-person.csv")
spirits_consumption = spirits_consumption[['Entity', 'Year', 'Indicator:Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol) - Beverage Types:Spirits']]
spirits_consumption = spirits_consumption.loc[spirits_consumption['Year'].isin([2012,2013,2014,2015,2016,2017])]
# Sort spirits consumption data by country and year and reset indexes and choose only country, and spirits consumption columns
spirits_consumption = spirits_consumption.rename(columns = {"Entity": "Country",  "Indicator:Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol) - Beverage Types:Spirits": "Spirits Consumption"})
spirits_consumption = spirits_consumption.sort_values(by=['Country', 'Year'])
spirits_consumption.insert(2, 'Sex', 'Both sexes')
spirits_consumption = spirits_consumption.reset_index(drop = True)
spirits_consumption = spirits_consumption[['Country', 'Year', 'Spirits Consumption']]
# Compute % of missing values in spirits consumption data frame before join
percentage_missing_values_spirits_consumption_before = ((float(spirits_consumption['Spirits Consumption'].isna().sum()))/spirits_consumption['Spirits Consumption'].count()) * 100.0
print("Percentage of Missing Values for Spirits Consumption Before Join: ", percentage_missing_values_spirits_consumption_before)
# Merge spirits consumption data with heart disease mortality data frame, Substituting all NaN values in spirits consumption column of main dataframe with mean values of corresponding years
heart_disease_death_rates['Spirits Consumption'] = spirits_consumption['Spirits Consumption']
heart_disease_death_rates['Spirits Consumption'] = heart_disease_death_rates['Spirits Consumption'].fillna(0.0)
for index, row in heart_disease_death_rates.iterrows():
    if row['Spirits Consumption'] == 0.0:
        spirits_consumption_col = heart_disease_death_rates[heart_disease_death_rates['Year'] == row['Year']]['Spirits Consumption'].dropna()
        spirits_consumption_mean = spirits_consumption_col.mean(axis = 0)
        heart_disease_death_rates.loc[index, 'Spirits Consumption'] = spirits_consumption_mean
# Compute % of missing values in spirits consumption after join
percentage_missing_values_spirits_consumption_after = ((float(heart_disease_death_rates['Spirits Consumption'].isna().sum()))/heart_disease_death_rates['Spirits Consumption'].count()) * 100.0
print("Percentage of Missing Values for Spirits Consumption After Join: ", percentage_missing_values_spirits_consumption_after)
heart_disease_death_rates

Percentage of Missing Values for Spirits Consumption Before Join:  0.0
Percentage of Missing Values for Spirits Consumption After Join:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI,Mean HDL Cholesterol,Low Physical Activity DALYs,Spirits Consumption
0,Afghanistan,2012,618.45,4.739260,22.900000,1.100000,87161.46,1.273202
1,Afghanistan,2013,607.69,4.766758,23.000000,1.100000,89240.38,1.244211
2,Afghanistan,2014,600.55,4.793179,23.200000,1.100000,91722.71,0.010000
3,Afghanistan,2015,597.02,4.810425,23.300000,1.100000,94787.74,1.203114
4,Afghanistan,2016,591.59,4.815518,23.400000,1.100000,97995.82,0.010000
...,...,...,...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756,1.169098,6607.48,1.617827
1364,Zimbabwe,2014,330.58,4.119858,24.120932,1.169612,6911.81,1.517327
1365,Zimbabwe,2015,329.57,4.124367,24.157551,1.168585,7196.88,1.503970
1366,Zimbabwe,2016,327.64,4.126087,24.128846,1.171152,7424.67,1.489531


### **Add smoking death percentage data**

The seventh step we will take is to import the [smoking death percentage data](https://ourworldindata.org/grapher/share-deaths-smoking), which is an age-standardized estimate (share of smoking deaths), and remove the columns we aren't going to use. This feature data comes from Our World in Data. The smoking death percentage (for each country's population) table includes data from 2012-2017 that has smoking death percentages for both sexes combined. There are no missing values in the smoking death percentage data before joining, so no initial imputation is required. After merging the smoking death percentage data with the heart disease mortality data frame, we can see that there are no missing values in the smoking death percentage column of the main data frame. Thus, no further imputation is required.

In [423]:
# Import smoking death percentage data, select and rename relevant columns, and choose all rows with years 2012-2017 (will only consider data for both sexes)
share_deaths_smoking = pd.read_csv("data/share-deaths-smoking.csv")
share_deaths_smoking = share_deaths_smoking[['Entity', 'Year', 'Deaths - Cause: All causes - Risk: Smoking - OWID - Sex: Both - Age: Age-standardized (Percent)']]
share_deaths_smoking = share_deaths_smoking.loc[share_deaths_smoking['Year'].isin([2012,2013,2014,2015,2016,2017])]
# Sort spirits consumption data by country and year and reset indexes and choose only country, and spirits consumption columns
share_deaths_smoking = share_deaths_smoking.rename(columns = {"Entity": "Country",  "Deaths - Cause: All causes - Risk: Smoking - OWID - Sex: Both - Age: Age-standardized (Percent)": "Smoking Death Percentage"})
share_deaths_smoking = share_deaths_smoking.sort_values(by=['Country', 'Year'])
share_deaths_smoking.insert(2, 'Sex', 'Both sexes')
share_deaths_smoking = share_deaths_smoking.reset_index(drop = True)
share_deaths_smoking = share_deaths_smoking[['Country', 'Year', "Smoking Death Percentage"]]
# Compute % of missing values in smoking deaths consumption data frame before join
percentage_missing_values_smoking_deaths_before = ((float(share_deaths_smoking["Smoking Death Percentage"].isna().sum()))/share_deaths_smoking["Smoking Death Percentage"].count()) * 100.0
print("Percentage of Missing Values for Share of Smoking Deaths Before Join: ", percentage_missing_values_smoking_deaths_before)
# Merge smoking death data with heart disease mortality data frame, Substituting all NaN values in smoking death percentage column of main dataframe with mean values of corresponding years
heart_disease_death_rates["Smoking Death Percentage"] = share_deaths_smoking["Smoking Death Percentage"]
# Compute % of missing values in smoking death percentage column after join
percentage_missing_values_smoking_deaths_after = ((float(heart_disease_death_rates["Smoking Death Percentage"].isna().sum()))/heart_disease_death_rates["Smoking Death Percentage"].count()) * 100.0
print("Percentage of Missing Values for Share of Smoking Deaths After Join: ", percentage_missing_values_smoking_deaths_after)
heart_disease_death_rates

Percentage of Missing Values for Share of Smoking Deaths Before Join:  0.0
Percentage of Missing Values for Share of Smoking Deaths After Join:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI,Mean HDL Cholesterol,Low Physical Activity DALYs,Spirits Consumption,Smoking Death Percentage
0,Afghanistan,2012,618.45,4.739260,22.900000,1.100000,87161.46,1.273202,8.38
1,Afghanistan,2013,607.69,4.766758,23.000000,1.100000,89240.38,1.244211,8.51
2,Afghanistan,2014,600.55,4.793179,23.200000,1.100000,91722.71,0.010000,8.51
3,Afghanistan,2015,597.02,4.810425,23.300000,1.100000,94787.74,1.203114,8.54
4,Afghanistan,2016,591.59,4.815518,23.400000,1.100000,97995.82,0.010000,8.58
...,...,...,...,...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756,1.169098,6607.48,1.617827,9.67
1364,Zimbabwe,2014,330.58,4.119858,24.120932,1.169612,6911.81,1.517327,9.80
1365,Zimbabwe,2015,329.57,4.124367,24.157551,1.168585,7196.88,1.503970,9.94
1366,Zimbabwe,2016,327.64,4.126087,24.128846,1.171152,7424.67,1.489531,10.06


### **Add raised blood pressure data**

The seventh step we will take is to import the [raised blood pressure data](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/raised-blood-pressure-(sbp-=140-or-dbp-=90)-(age-standardized-estimate)), which is an age-standardized estimate (% of population with raised blood pressure). We define raised blood pressure as SBP (systolic blood pressure)>= 140 or DBP (diastolic blood pressure) >= 90 and remove the columns we aren't going to use. This feature data comes from the World Health Organization. The raised blood pressure percentage (for each country's population) table includes data from 2012-2015 that has raised blood pressure percentages for both sexes combined. There are no missing values in the raised blood pressure percentage data before joining, so no initial imputation is required. After merging the raised blood pressure percentage data with the heart disease mortality data frame and substituting all NaN blood pressure values for 2012-2015 with mean of the corresponding year and 2016-2017 values with the mean for the corresponding country, we can see that there are no missing values in the raised blood pressure percentage column of the main data frame. Thus, no further imputation will be required at that point.

In [424]:
# Ndenko: raised blood pressure, Deaths from diabetes by type, Health expenditure as % of GDP
# Import raised blood pressure data, select and rename relevant columns, and choose all rows that have both sexes. Will have to fill in missing 2016 and 2017 data later.
blood_pressure = pd.read_csv('data/bld pressure.csv')
blood_pressure = blood_pressure[['Location', 'Period', 'Dim1','Dim2', 'FactValueNumeric']]
blood_pressure = blood_pressure.rename(columns = {"Location": "Country", "Period": "Year", "Dim1":"Sex", "Dim2":"Age", "FactValueNumeric": "Raised Blood Pressure Percentage"})
# # Sort raised blood pressure data by country and year and reset indexes and choose only country, year, and raised blood pressure percentage columns
blood_pressure = blood_pressure.sort_values(by=['Country', 'Year'])
blood_pressure = blood_pressure[blood_pressure['Sex'] == "Both sexes"]
blood_pressure = blood_pressure[['Country', 'Year', 'Raised Blood Pressure Percentage']]
blood_pressure = blood_pressure.reset_index(drop = True)
# Compute % of missing values in smoking deaths consumption data frame before join
percentage_missing_values_raised_blood_pressure_before = ((float(blood_pressure["Raised Blood Pressure Percentage"].isna().sum()))/blood_pressure["Raised Blood Pressure Percentage"].count()) * 100.0
print("Percentage of Missing Values for Raised Blood Pressure Before Join and Initial Imputation: ", percentage_missing_values_raised_blood_pressure_before)
# Fill all missing values inside the original blood pressure data frame with the average blood pressure of that year and confirm no more missing values in blood pressure data frame after initial imputation
blood_pressure['Raised Blood Pressure Percentage'] = blood_pressure['Raised Blood Pressure Percentage'].fillna(0.0)
for index, row in blood_pressure.iterrows():
    if row['Raised Blood Pressure Percentage'] == 0.0:
        current_year_bp_rows = blood_pressure[blood_pressure['Year'] == row['Year']]
        blood_pressure_mean  = current_year_bp_rows['Raised Blood Pressure Percentage'].mean(axis = 0)
        blood_pressure.loc[index, 'Raised Blood Pressure Percentage'] = blood_pressure_mean
percentage_missing_values_raised_blood_pressure_before_2 = ((float(blood_pressure["Raised Blood Pressure Percentage"].isna().sum()))/blood_pressure["Raised Blood Pressure Percentage"].count()) * 100.0
print("Percentage of Missing Values for Raised Blood Pressure Before Join and After Initial Imputation: ", percentage_missing_values_raised_blood_pressure_before_2)
# Merge raised blood pressure data with heart disease mortality data frame, substitute all NaN blood pressure values for 2012-2015 with 0.0 and eventually mean blood pressure of that particular year
heart_disease_death_rates["Raised Blood Pressure Percentage"] = blood_pressure["Raised Blood Pressure Percentage"]
heart_disease_death_rates["Raised Blood Pressure Percentage"] = heart_disease_death_rates["Raised Blood Pressure Percentage"].fillna(0.0)
for index, row in heart_disease_death_rates.iterrows():
    if row['Year'] < 2016 and row['Raised Blood Pressure Percentage'] == 0.0:
        current_year_bp_rows2 = heart_disease_death_rates[heart_disease_death_rates['Year'] == row['Year']]
        blood_pressure_mean2  = current_year_bp_rows['Raised Blood Pressure Percentage'].mean(axis = 0)
        heart_disease_death_rates.loc[index, 'Raised Blood Pressure Percentage'] = blood_pressure_mean2

# Substitute all 2016 and 2017 blood pressure values (0.0) with mean raised blood pressure for corresponding countries (In Progress)
#for country in heart_disease_death_rates['Country'].unique():
    #country_mean_blood_pressure_nonnull_rows = heart_disease_death_rates[(heart_disease_death_rates['Country'] == country) & (heart_disease_death_rates['Raised Blood Pressure Percentage'] != 0.0)]
    #country_mean_blood_pressure_nonnull = country_mean_blood_pressure_nonnull_rows['Raised Blood Pressure Percentage'].mean(axis = 0)
    #country_mean_blood_pressure_null_row_indexes = heart_disease_death_rates[(heart_disease_death_rates['Country'] == country) & (heart_disease_death_rates['Raised Blood Pressure Percentage'] != 0.0)].index
    #for i in country_mean_blood_pressure_null_row_indexes:
        
#print(heart_disease_death_rates[(heart_disease_death_rates['Country'] == 'Zimbabwe') & (heart_disease_death_rates['Raised Blood Pressure Percentage'] != 0.0)].index)

# Compute % of missing values in smoking death percentage column after join
percentage_missing_values_raised_blood_pressure_after = ((float(heart_disease_death_rates["Raised Blood Pressure Percentage"].isna().sum()))/heart_disease_death_rates["Raised Blood Pressure Percentage"].count()) * 100.0
print("Percentage of Missing Values for Raised Blood Pressure After Join and Imputation: ", percentage_missing_values_raised_blood_pressure_after)
heart_disease_death_rates

Percentage of Missing Values for Raised Blood Pressure Before Join and Initial Imputation:  2.094240837696335
Percentage of Missing Values for Raised Blood Pressure Before Join and After Initial Imputation:  0.0
Index([1362, 1363, 1364, 1365], dtype='int64')
Percentage of Missing Values for Raised Blood Pressure After Join and Imputation:  0.0


,Country,Year,Heart Disease Mortality Rate (Deaths per 100K people),% of population with anxiety disorders,Mean BMI,Mean HDL Cholesterol,Low Physical Activity DALYs,Spirits Consumption,Smoking Death Percentage,Raised Blood Pressure Percentage
0,Afghanistan,2012,618.45,4.739260,22.900000,1.100000,87161.46,1.273202,8.38,30.500000
1,Afghanistan,2013,607.69,4.766758,23.000000,1.100000,89240.38,1.244211,8.51,30.500000
2,Afghanistan,2014,600.55,4.793179,23.200000,1.100000,91722.71,0.010000,8.51,30.600000
3,Afghanistan,2015,597.02,4.810425,23.300000,1.100000,94787.74,1.203114,8.54,30.600000
4,Afghanistan,2016,591.59,4.815518,23.400000,1.100000,97995.82,0.010000,8.58,29.700000
...,...,...,...,...,...,...,...,...,...,...
1363,Zimbabwe,2013,331.36,4.113508,24.191756,1.169098,6607.48,1.617827,9.67,24.297181
1364,Zimbabwe,2014,330.58,4.119858,24.120932,1.169612,6911.81,1.517327,9.80,24.297181
1365,Zimbabwe,2015,329.57,4.124367,24.157551,1.168585,7196.88,1.503970,9.94,24.297181
1366,Zimbabwe,2016,327.64,4.126087,24.128846,1.171152,7424.67,1.489531,10.06,0.000000


In [425]:

diabetes_deaths_both_types = pd.read_csv('data/deaths-from-diabetes-by-type.csv')
diabetes_deaths_both_types = diabetes_deaths_both_types[['Entity', 'Year','Deaths - Diabetes mellitus type 1 - Sex: Both - Age: All Ages (Number)', 'Deaths - Diabetes mellitus type 2 - Sex: Both - Age: All Ages (Number)']]
diabetes_deaths_both_types = diabetes_deaths_both_types.rename(columns = {"Entity": "Country", "Deaths - Diabetes mellitus type 1 - Sex: Both - Age: All Ages (Number)":"Type 1 Deaths","Deaths - Diabetes mellitus type 2 - Sex: Both - Age: All Ages (Number)":"Type 2 Deaths"})
diabetes_deaths_both_types = diabetes_deaths_both_types.sort_values(by=['Country', 'Year'])
# Since the source indicated both sexes, and all ages, add columns to indicate that
# create the values we need to add 2 long arrays
both_sexes = np.full(len(diabetes_deaths_both_types), "Both sexes")
all_ages = np.full(len(diabetes_deaths_both_types), "All ages")
# add those values as new columns
diabetes_deaths_both_types['Sex'] = both_sexes
diabetes_deaths_both_types['Age'] = all_ages
# rearrange the order to match the format of previous tables
desired_order = ['Country','Year','Sex','Age','Type 1 Deaths','Type 2 Deaths']
diabetes_deaths_both_types = diabetes_deaths_both_types[desired_order]
# this data is from 1990 - 2019, filter it for only 2012-2017
diabetes_deaths_both_types = diabetes_deaths_both_types[diabetes_deaths_both_types["Year"].between(2012, 2017)]
diabetes_deaths_both_types = diabetes_deaths_both_types.reset_index(drop = True)
diabetes_deaths_both_types

,Country,Year,Sex,Age,Type 1 Deaths,Type 2 Deaths
0,Afghanistan,2012,Both sexes,All ages,435,3247
1,Afghanistan,2013,Both sexes,All ages,445,3381
2,Afghanistan,2014,Both sexes,All ages,450,3535
3,Afghanistan,2015,Both sexes,All ages,454,3689
4,Afghanistan,2016,Both sexes,All ages,462,3887
...,...,...,...,...,...,...
1363,Zimbabwe,2013,Both sexes,All ages,104,2902
1364,Zimbabwe,2014,Both sexes,All ages,105,2988
1365,Zimbabwe,2015,Both sexes,All ages,109,3067
1366,Zimbabwe,2016,Both sexes,All ages,113,3146


In [426]:
# filter the table down to just type 1 deaths
diabetes_deaths_type1 = diabetes_deaths_both_types[['Country','Year','Sex','Age','Type 1 Deaths']]
diabetes_deaths_type1

,Country,Year,Sex,Age,Type 1 Deaths
0,Afghanistan,2012,Both sexes,All ages,435
1,Afghanistan,2013,Both sexes,All ages,445
2,Afghanistan,2014,Both sexes,All ages,450
3,Afghanistan,2015,Both sexes,All ages,454
4,Afghanistan,2016,Both sexes,All ages,462
...,...,...,...,...,...
1363,Zimbabwe,2013,Both sexes,All ages,104
1364,Zimbabwe,2014,Both sexes,All ages,105
1365,Zimbabwe,2015,Both sexes,All ages,109
1366,Zimbabwe,2016,Both sexes,All ages,113


In [427]:
# filter the table down to just type 1 deaths
diabetes_deaths_type2 = diabetes_deaths_both_types[['Country','Year','Sex','Age','Type 2 Deaths']]
diabetes_deaths_type2

,Country,Year,Sex,Age,Type 2 Deaths
0,Afghanistan,2012,Both sexes,All ages,3247
1,Afghanistan,2013,Both sexes,All ages,3381
2,Afghanistan,2014,Both sexes,All ages,3535
3,Afghanistan,2015,Both sexes,All ages,3689
4,Afghanistan,2016,Both sexes,All ages,3887
...,...,...,...,...,...
1363,Zimbabwe,2013,Both sexes,All ages,2902
1364,Zimbabwe,2014,Both sexes,All ages,2988
1365,Zimbabwe,2015,Both sexes,All ages,3067
1366,Zimbabwe,2016,Both sexes,All ages,3146


In [428]:
health_expenditure_as_percent_gdp = pd.read_csv('data/health expenditure as percentage of GDP.csv')
health_expenditure_as_percent_gdp = health_expenditure_as_percent_gdp[['Location', 'Period', 'FactValueNumeric']]
health_expenditure_as_percent_gdp = health_expenditure_as_percent_gdp.rename(columns = {"Location": "Country", "Period": "Year", "FactValueNumeric": "Healthcare Expenditure (% of GDP)"})
health_expenditure_as_percent_gdp = health_expenditure_as_percent_gdp.sort_values(by=['Country', 'Year'])
health_expenditure_as_percent_gdp = health_expenditure_as_percent_gdp.reset_index(drop = True)

health_expenditure_as_percent_gdp

,Country,Year,Healthcare Expenditure (% of GDP)
0,Afghanistan,2012,7.90
1,Afghanistan,2013,8.81
2,Afghanistan,2014,9.53
3,Afghanistan,2015,10.11
4,Afghanistan,2016,11.82
...,...,...,...
1127,Zimbabwe,2013,7.11
1128,Zimbabwe,2014,8.13
1129,Zimbabwe,2015,7.45
1130,Zimbabwe,2016,7.47
